# 5on5 Shot Data
The following mines NHL shot data from the 2019-20 regular season in all 5on5 scenarios (5 offensive and 5 defensive players)

In [2]:
import requests
import pickle
import json

Functions needed to filter non 5on5 data

In [3]:
def calc_even_strength_period(penalty_minutes, period, seconds):
    new_seconds = penalty_minutes * 60 + seconds
    if new_seconds >= 1200:
        return period + 1
    return period

def calc_even_strength_seconds(penalty_minutes, period, seconds):
    new_seconds = penalty_minutes * 60 + seconds
    return new_seconds % 1200

def calc_seconds(time):
    min = int(time[0: time.index(':')])
    sec = int(time[time.index(':') + 1:])
    return min * 60 + sec

### Start building api call data

In [4]:
# Set up the API call variables
game_data = []
year = '2019'
season_type = '02' 
max_game_ID = 1271
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', 
    "Upgrade-Insecure-Requests": "1",
    "DNT": "1",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate"
}


print('https://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(762).zfill(4)+'/feed/live')

https://statsapi.web.nhl.com/api/v1/game/2019020762/feed/live


### Loop through each game and collect data

In [5]:
# Loop over the counter and format the API call
for i in range(1,max_game_ID):
    r = requests.get(url='https://statsapi.web.nhl.com/api/v1/game/'
        + year + season_type +str(i).zfill(4)+'/feed/live')
    d = r.json()

    # Initialize penalty information
    penalty_info = {
        d['gameData']['teams']['home']['name'] : {
            "num_in_box" : 0,
            "even_strength_period": 0,
            "even_strength_seconds" : 0,
            "double_minor" : False,
            "major" : False
        },
        d['gameData']['teams']['away']['name'] : {
            "num_in_box" : 0,
            "even_strength_period": 0,
            "even_strength_seconds" : 0,
            "double_minor" : False,
            "major" : False
        }
    }
    print(d['gameData']['teams']['home']['name'] + ' vs ' + d['gameData']['teams']['away']['name'] + ' : ' + str(d['gamePk']))

    even_strength = True    # Start the game off on even stength
    for data in d:
        if 'liveData' not in data:
            continue

        single_game = d[data]['plays']['allPlays']
        for play in single_game:
            period = play['about']['period']                    # current period
            time = calc_seconds(play['about']['periodTime'])    # current time in seconds

            # update penalty information when a new penalty occurs
            # sometimes penalty will be a double minor
            if play['result']['eventTypeId'] == 'PENALTY' and (play['result']['penaltySeverity'] == 'Bench Minor' or play['result']['penaltySeverity'] == 'Minor'):
                team = play['team']['name']
                penalty_minutes = play['result']['penaltyMinutes']
                
                penalty_info[team]['num_in_box'] += 1
                penalty_info[team]['even_strength_period'] = calc_even_strength_period(penalty_minutes, period, time)
                penalty_info[team]['even_strength_seconds'] = calc_even_strength_seconds(penalty_minutes, period, time)
                even_strength = False

                if penalty_minutes == 4:
                    penalty_info[team]['double_minor'] = True

                # print('\nNew Penalty:')
                # print(period)
                # print(time)
                # print(penalty_info[team])
            
            # sometimes penalty will be a major (5 minute penalty)
            elif play['result']['eventTypeId'] == 'PENALTY' and play['result']['penaltySeverity'] == 'Major' and play['result']['secondaryType'] != 'Fighting':
                team = play['team']['name']
                penalty_minutes = play['result']['penaltyMinutes']
                
                penalty_info[team]['num_in_box'] += 1
                penalty_info[team]['even_strength_period'] = calc_even_strength_period(penalty_minutes, period, time)
                penalty_info[team]['even_strength_seconds'] = calc_even_strength_seconds(penalty_minutes, period, time)
                even_strength = False
                penalty_info[team]['major'] = True

                # print('\nNew Penalty:')
                # print(period)
                # print(time)
                # print(penalty_info[team])
            
            # if a goal is scored, remove a person from penalty box
            # however, if penalty was a double minor then only adjust 'even_strength_time'
            # if penalty is a major, only adjust 'num_in_box'
            elif play['result']['eventTypeId'] == 'GOAL' and play['result']['strength']['code'] == 'PPG':
                scoring_team = play['team']['name']
                for team in penalty_info:
                    if team != scoring_team:
                        # print("\nPowerPlay Goal at " + str(calc_seconds(play['about']['periodTime'])))
                        # for double minors
                        if penalty_info[team]['double_minor']:
                            new_expirary_time = (time + 120) % 1200     # calculate new time
                            if penalty_info[team]['even_strength_seconds'] < new_expirary_time:
                                # time left on penalty less than 120 seconds, let man out of box
                                penalty_info[team]['double_minor'] = False
                                penalty_info[team]['num_in_box'] -= 1
                                if penalty_info[team]['num_in_box'] == 0:
                                    even_strength = True
                            else:
                                # set new penalty time
                                penalty_info[team]['even_strength_seconds'] = new_expirary_time
                                penalty_info[team]['double_minor'] = False
                            
                            # print('powerplay ending at: ' + str(penalty_info[team]['even_strength_period']) + ' : ' + str(penalty_info[team]['even_strength_seconds']))
                            # print(penalty_info[team])
                            # if even_strength:
                            #     print('at even strength')
                        
                        # for major penalties
                        elif penalty_info[team]['major'] and penalty_info[team]['num_in_box'] > 1:
                            penalty_info[team]['num_in_box'] -= 1

                        # for regular bench minors
                        elif not penalty_info[team]['major'] and not penalty_info[team]['double_minor']:      
                            penalty_info[team]['num_in_box'] -= 1
                            if penalty_info[team]['num_in_box'] == 0:
                                even_strength = True

            # check if penalty expired
            elif not even_strength:
                both_teams_expired = True
                for team in penalty_info:
                    if penalty_info[team]['even_strength_period'] > period:
                        both_teams_expired = False
                    elif penalty_info[team]['even_strength_period'] == period and penalty_info[team]['even_strength_seconds'] > time:
                        both_teams_expired = False
                    else:
                        penalty_info[team]['num_in_box'] = 0
                        penalty_info[team]['double_minor'] = False
                        penalty_info[team]['major'] = False
                even_strength = both_teams_expired
                if even_strength:
                    game_data.append(play)

            # if play is at even strength, append list
            elif even_strength:
                game_data.append(play)

Toronto Maple Leafs vs Ottawa Senators : 2019020001
St. Louis Blues vs Washington Capitals : 2019020002
Edmonton Oilers vs Vancouver Canucks : 2019020003
Vegas Golden Knights vs San Jose Sharks : 2019020004
Tampa Bay Lightning vs Florida Panthers : 2019020005
New York Rangers vs Winnipeg Jets : 2019020006
Pittsburgh Penguins vs Buffalo Sabres : 2019020007
Carolina Hurricanes vs Montréal Canadiens : 2019020008
Nashville Predators vs Minnesota Wild : 2019020009
Dallas Stars vs Boston Bruins : 2019020010
Colorado Avalanche vs Calgary Flames : 2019020011
Anaheim Ducks vs Arizona Coyotes : 2019020012
New Jersey Devils vs Winnipeg Jets : 2019020013
New York Islanders vs Washington Capitals : 2019020014
Columbus Blue Jackets vs Toronto Maple Leafs : 2019020015
Philadelphia Flyers vs Chicago Blackhawks : 2019020016
San Jose Sharks vs Vegas Golden Knights : 2019020017
Buffalo Sabres vs New Jersey Devils : 2019020018
Toronto Maple Leafs vs Montréal Canadiens : 2019020019
Ottawa Senators vs New Y

Montréal Canadiens vs Toronto Maple Leafs : 2019020163
Tampa Bay Lightning vs Nashville Predators : 2019020164
Philadelphia Flyers vs Columbus Blue Jackets : 2019020165
Dallas Stars vs Pittsburgh Penguins : 2019020166
Minnesota Wild vs Los Angeles Kings : 2019020167
Winnipeg Jets vs Calgary Flames : 2019020168
Colorado Avalanche vs Anaheim Ducks : 2019020169
Edmonton Oilers vs Florida Panthers : 2019020170
Detroit Red Wings vs St. Louis Blues : 2019020171
Ottawa Senators vs San Jose Sharks : 2019020172
New York Islanders vs Philadelphia Flyers : 2019020173
New York Rangers vs Boston Bruins : 2019020174
Chicago Blackhawks vs Los Angeles Kings : 2019020175
Vegas Golden Knights vs Anaheim Ducks : 2019020176
Buffalo Sabres vs Arizona Coyotes : 2019020177
Vancouver Canucks vs Florida Panthers : 2019020178
Boston Bruins vs San Jose Sharks : 2019020179
Toronto Maple Leafs vs Washington Capitals : 2019020180
New York Rangers vs Tampa Bay Lightning : 2019020181
Pittsburgh Penguins vs Philadelph

Nashville Predators vs Winnipeg Jets : 2019020325
Chicago Blackhawks vs Carolina Hurricanes : 2019020326
Dallas Stars vs Vancouver Canucks : 2019020327
Calgary Flames vs Colorado Avalanche : 2019020328
Vegas Golden Knights vs Toronto Maple Leafs : 2019020329
San Jose Sharks vs Edmonton Oilers : 2019020330
Montréal Canadiens vs Ottawa Senators : 2019020331
New York Rangers vs Washington Capitals : 2019020332
Boston Bruins vs Buffalo Sabres : 2019020333
Florida Panthers vs Anaheim Ducks : 2019020334
New York Islanders vs Pittsburgh Penguins : 2019020335
Carolina Hurricanes vs Philadelphia Flyers : 2019020336
Columbus Blue Jackets vs Detroit Red Wings : 2019020337
St. Louis Blues vs Calgary Flames : 2019020338
Nashville Predators vs Vancouver Canucks : 2019020339
Minnesota Wild vs Colorado Avalanche : 2019020340
Chicago Blackhawks vs Tampa Bay Lightning : 2019020341
Dallas Stars vs Winnipeg Jets : 2019020342
Arizona Coyotes vs Toronto Maple Leafs : 2019020343
Vegas Golden Knights vs San J

Buffalo Sabres vs Nashville Predators : 2019020487
Tampa Bay Lightning vs Boston Bruins : 2019020488
Florida Panthers vs New York Islanders : 2019020489
Pittsburgh Penguins vs Columbus Blue Jackets : 2019020490
Detroit Red Wings vs Winnipeg Jets : 2019020491
St. Louis Blues vs Vegas Golden Knights : 2019020492
Minnesota Wild vs Edmonton Oilers : 2019020493
Calgary Flames vs Toronto Maple Leafs : 2019020494
Arizona Coyotes vs Chicago Blackhawks : 2019020495
Vancouver Canucks vs Carolina Hurricanes : 2019020496
Anaheim Ducks vs Los Angeles Kings : 2019020497
San Jose Sharks vs New York Rangers : 2019020498
Dallas Stars vs Vegas Golden Knights : 2019020499
Colorado Avalanche vs New Jersey Devils : 2019020500
Ottawa Senators vs Columbus Blue Jackets : 2019020501
New York Islanders vs Buffalo Sabres : 2019020502
Calgary Flames vs Carolina Hurricanes : 2019020503
Anaheim Ducks vs New York Rangers : 2019020504
Nashville Predators vs Dallas Stars : 2019020505
Montréal Canadiens vs Detroit Red 

Ottawa Senators vs Tampa Bay Lightning : 2019020648
New Jersey Devils vs Colorado Avalanche : 2019020649
Arizona Coyotes vs Philadelphia Flyers : 2019020650
Vancouver Canucks vs New York Rangers : 2019020651
Los Angeles Kings vs Nashville Predators : 2019020652
Washington Capitals vs San Jose Sharks : 2019020653
Pittsburgh Penguins vs Florida Panthers : 2019020654
Carolina Hurricanes vs Tampa Bay Lightning : 2019020655
Minnesota Wild vs Calgary Flames : 2019020656
Chicago Blackhawks vs Detroit Red Wings : 2019020657
Anaheim Ducks vs Nashville Predators : 2019020658
Toronto Maple Leafs vs Edmonton Oilers : 2019020659
Montréal Canadiens vs Winnipeg Jets : 2019020660
New York Islanders vs Colorado Avalanche : 2019020661
Los Angeles Kings vs Columbus Blue Jackets : 2019020662
Tampa Bay Lightning vs Vancouver Canucks : 2019020663
Florida Panthers vs Arizona Coyotes : 2019020664
New Jersey Devils vs New York Islanders : 2019020665
New York Rangers vs Colorado Avalanche : 2019020666
Washingto

Carolina Hurricanes vs Vancouver Canucks : 2019020808
Toronto Maple Leafs vs Florida Panthers : 2019020809
New York Rangers vs Dallas Stars : 2019020810
Detroit Red Wings vs Philadelphia Flyers : 2019020811
Boston Bruins vs Vancouver Canucks : 2019020812
Buffalo Sabres vs Colorado Avalanche : 2019020813
Tampa Bay Lightning vs Vegas Golden Knights : 2019020814
New Jersey Devils vs Montréal Canadiens : 2019020815
New York Islanders vs Dallas Stars : 2019020816
Washington Capitals vs Los Angeles Kings : 2019020817
Columbus Blue Jackets vs Florida Panthers : 2019020818
Ottawa Senators vs Anaheim Ducks : 2019020819
St. Louis Blues vs Carolina Hurricanes : 2019020820
Minnesota Wild vs Chicago Blackhawks : 2019020821
Winnipeg Jets vs Nashville Predators : 2019020822
Calgary Flames vs San Jose Sharks : 2019020823
Arizona Coyotes vs Edmonton Oilers : 2019020824
New York Rangers vs Toronto Maple Leafs : 2019020825
Chicago Blackhawks vs Boston Bruins : 2019020826
Buffalo Sabres vs Detroit Red Win

Tampa Bay Lightning vs Toronto Maple Leafs : 2019020970
New York Islanders vs New York Rangers : 2019020971
Philadelphia Flyers vs San Jose Sharks : 2019020972
Washington Capitals vs Winnipeg Jets : 2019020973
Carolina Hurricanes vs Dallas Stars : 2019020974
Detroit Red Wings vs New Jersey Devils : 2019020975
St. Louis Blues vs Chicago Blackhawks : 2019020976
Nashville Predators vs Ottawa Senators : 2019020977
Minnesota Wild vs Columbus Blue Jackets : 2019020978
Arizona Coyotes vs Florida Panthers : 2019020979
Anaheim Ducks vs Edmonton Oilers : 2019020980
Colorado Avalanche vs Buffalo Sabres : 2019020981
Vegas Golden Knights vs Edmonton Oilers : 2019020982
Los Angeles Kings vs Pittsburgh Penguins : 2019020983
Boston Bruins vs Dallas Stars : 2019020984
Montréal Canadiens vs New York Rangers : 2019020985
Tampa Bay Lightning vs Chicago Blackhawks : 2019020986
Florida Panthers vs Toronto Maple Leafs : 2019020987
Ottawa Senators vs Vancouver Canucks : 2019020988
Detroit Red Wings vs Minneso

Los Angeles Kings vs Montréal Canadiens : 2019021131
Ottawa Senators vs Edmonton Oilers : 2019021132
New York Rangers vs Pittsburgh Penguins : 2019021133
Vancouver Canucks vs Tampa Bay Lightning : 2019021134
Arizona Coyotes vs Vegas Golden Knights : 2019021135
Anaheim Ducks vs Boston Bruins : 2019021136
Toronto Maple Leafs vs New York Islanders : 2019021137
Florida Panthers vs Buffalo Sabres : 2019021138
New Jersey Devils vs Calgary Flames : 2019021139
Carolina Hurricanes vs St. Louis Blues : 2019021140
Columbus Blue Jackets vs Washington Capitals : 2019021141
Nashville Predators vs Colorado Avalanche : 2019021142
Chicago Blackhawks vs Minnesota Wild : 2019021143
Los Angeles Kings vs Boston Bruins : 2019021144
San Jose Sharks vs Montréal Canadiens : 2019021145
Pittsburgh Penguins vs New York Rangers : 2019021146
Washington Capitals vs Ottawa Senators : 2019021147
Winnipeg Jets vs Minnesota Wild : 2019021148
Dallas Stars vs Philadelphia Flyers : 2019021149
Edmonton Oilers vs Tampa Bay L

In [12]:
events = ['Goal']
even_goals = 0
for play in game_data:
    if play['result']['event'] in events:
        even_goals += 1
        #print(play['result']['event'] + ' : ' + play['about']['ordinalNum'] + ' : ' + str(calc_seconds(play['about']['periodTime'])))
print('numPlays: ' + str(len(game_data)))
print('Even Strength Goals: ' + str(even_goals))


numPlays: 284248
Even Strength Goals: 5077


In [9]:
with open('./input/'+year+'fiveVfive.pkl', 'wb') as f:
    pickle.dump(game_data, f, pickle.HIGHEST_PROTOCOL)